In [4]:
!pip install requests-html


  Obtaining dependency information for requests-html from https://files.pythonhosted.org/packages/24/bc/a4380f09bab3a776182578ce6b2771e57259d0d4dbce178205779abdc347/requests_html-0.10.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyquery from https://files.pythonhosted.org/packages/36/b7/f7ccf9e52e2817e1265d3719c600fa4ef33c07de4d5ef0ced3f43ab1cef2/pyquery-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for fake-useragent from https://files.pythonhosted.org/packages/e4/99/60d8cf1b26938c2e0a57e232f7f15641dfcd6f8deda454d73e4145910ff6/fake_useragent-1.5.1-py3-none-any.whl.metadata
  Obtaining dependency information for parse from https://files.pythonhosted.org/packages/d0/31/ba45bf0b2aa7898d81cbbfac0e88c267befb59ad91a19e36e1bc5578ddb1/parse-1.20.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for bs4 from https://files.pythonhosted.org/packages/51/bb/bf7aab772a159614954d84aa832c129624ba6c32faa559dfb200a534e50b/bs4-0.0.2-py2.py3-non

In [8]:
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import os
import psycopg2
from psycopg2 import sql
from psycopg2.extras import execute_values
import requests
from requests_html import HTMLSession


def get_cookie_http():
    url = "https://rimafinance.com"
    response = requests.get(url, timeout=55, verify=False)
    cookies = response.cookies
    for cookie in cookies:
        if cookie.name == "PHPSESSID":
            return cookie.value


# If the cookie is set via JavaScript:
def get_cookie_js():
    session = HTMLSession()
    url = "https://rimafinance.com"
    response = session.get(url, timeout=55, verify=False)
    response.html.render()  # This will execute the JavaScript
    cookies = session.cookies
    for cookie in cookies:
        if cookie.name == "PHPSESSID":
            return cookie.value


def ingest_data():
    cookie_value = get_cookie_http() or get_cookie_js()
    cookies = {
        "PHPSESSID": cookie_value,
    }
    headers = {
        "Accept": "*/*",
        "Accept-Language": "en,en-US;q=0.9,de;q=0.8,sm;q=0.7,fa;q=0.6",
        "Cache-Control": "no-cache",
        "Connection": "keep-alive",
        # 'Cookie': 'PHPSESSID=elklm86dd23r97s1mt89usc570',
        "Pragma": "no-cache",
        "Referer": "https://www.rimafinance.com/?size=500",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1",
        "X-Requested-With": "XMLHttpRequest",
    }

    response = requests.get(
        "https://www.rimafinance.com/ajax/all_prices",
        headers=headers,
        cookies=cookies,
        timeout=55,
        verify=False,
    )
    return response

In [9]:
ingest_data().text

/home/levi/miniconda3/envs/mikasa/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rimafinance.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/levi/miniconda3/envs/mikasa/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.rimafinance.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/levi/miniconda3/envs/mikasa/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.rimafinance.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warni

'{"meta":{"time":1719741603849,"time_formatted":"","fiyat_yayini":"web_socket","fiyat_guncelleme":2000},"data":{"ALTIN":{"code":"ALTIN","alis":"2466.92","satis":"2480.37","tarih":"30-06-2024 13:00:02","dir":{"alis_dir":"up","satis_dir":"up"},"dusuk":2480.37,"yuksek":2480.37,"kapanis":2466.92},"USDIRRt":{"code":"USDIRRt","alis":"61800","satis":"61800","tarih":"30-06-2024 13:00:03","dir":{"alis_dir":"down","satis_dir":"down"},"dusuk":60800,"yuksek":61800,"kapanis":60800},"EURUSD":{"code":"EURUSD","alis":"1.0682","satis":"1.0715","tarih":"30-06-2024 13:00:02","dir":{"alis_dir":"up","satis_dir":"up"},"dusuk":1.0715,"yuksek":1.0715,"kapanis":1.0682},"XAUUSD":{"code":"XAUUSD","alis":"2326.3","satis":"2326.9","tarih":"30-06-2024 13:00:02","dir":{"alis_dir":"up","satis_dir":"up"},"dusuk":0,"yuksek":2326.9,"kapanis":2326.3},"USDIRR":{"code":"USDIRR","alis":"61900","satis":"62250","tarih":"30-06-2024 13:00:03","dir":{"alis_dir":"up","satis_dir":"up"},"dusuk":61300,"yuksek":62300,"kapanis":61020}